## Trying out basic FIRE calculations

In [2]:
SIP = input()
SIP

 40


'40'

In [4]:
!pip install ipywidgets


In [3]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbclassic nbconvert notebook run server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


# Taking SIP input

In [13]:
import ipywidgets as widgets
# from IPython.display import display, VBox, HBox
from IPython.display import display
from ipywidgets import VBox, HBox 


# Create slider and manual input
sip_slider = widgets.IntSlider(
    value=20000,
    min=1000,
    max=200000,
    step=1000,
    description='Monthly SIP (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

sip_input = widgets.BoundedIntText(
    value=20000,
    min=1000,
    max=200000,
    step=1000,
    description='Enter SIP (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

# Display output label
sip_display = widgets.Label()

# Sync slider and manual input
widgets.jslink((sip_slider, 'value'), (sip_input, 'value'))

# Update display label on change
def update_display(change):
    sip_display.value = f"👉 Your Monthly SIP is set to ₹{change['new']:,}"

sip_slider.observe(update_display, names='value')

# Display the UI
ui = VBox([
    HBox([sip_slider, sip_input]),
    sip_display
])
display(ui)

# Trigger initial display
sip_display.value = f"👉 Your Monthly SIP is set to ₹{sip_slider.value:,}"
